In [ ]:
from alpha.notebook import *

await init_notebook()

In [2]:
async def scan(end):
    results = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        end = tf.floor(arrow.get(end), FrameType.MIN30)
        start = tf.shift(end, -40, FrameType.MIN30)
        bars = await sec.load_bars(start, end, FrameType.MIN30)
        if len(bars) <40 or np.count_nonzero(np.isfinite(bars["close"])) < 0.9 < len(bars):
            continue

        vf = top_volume_direction(bars, n= 40)

        day_end = end.date()
        day_start = tf.day_shift(day_end, -1)   
        try:
            day_bars = await sec.load_bars(day_start, day_end, FrameType.DAY)
            c1, c0 = day_bars["close"]
        except Exception:
            pass
        row = [sec.display_name, code, c0/c1-1]
        row.extend(vf)

        results.append(row)

    df = pd.DataFrame(results, columns=["name", "code", "adv", "vf1", "vf2", "vf3"])
    df["vsum"] = df.vf1 + df.vf2 + df.vf3
    
    return df

In [3]:
def query(df, adv=0.1, vsum=6, up_ratio = 10, down_ratio = -3):
    df_ = df
    df_ = df_[df_.vsum>vsum]
    df_ = df_[(df_.vf1 > down_ratio) & (df_.vf2 > down_ratio) & (df_.vf3 > down_ratio)]
    df_ = df_[(df_.vf1 > up_ratio) | (df_.vf2 > up_ratio) | (df_.vf3 > up_ratio)]
    df_ = df_[df_.adv < adv]
    df_ = df_.sort_values("adv")
    
    df_["adv"] = df_.adv.apply(lambda x: f"{x:.1%}")
    return df_

In [4]:
df = await scan("2021-09-16 15:00")

In [6]:
query(df)

,name,code,adv,vf1,vf2,vf3,vsum
1647,金明精机,300281.XSHE,1.3%,2.864361,-1.690417,10.085858,11.259802
2186,锦盛新材,300849.XSHE,3.0%,-2.253884,2.329375,10.046038,10.121529
216,亚太实业,000691.XSHE,3.9%,2.655571,10.449416,-2.133926,10.971061
2343,漱玉平民,301017.XSHE,6.4%,12.969839,-1.899223,-2.131899,8.938718
1941,万里马,300591.XSHE,7.1%,13.258433,-1.991539,-1.785818,9.481076


In [ ]:
import os
os.makedirs("/tmp/byVolume/0914")
for row in query(df).to_records():
    code = row["code"]
    cs = Candlestick()
    await cs.plot(code, "2021-09-15 15:00", save_to="/tmp/byVolume")

In [ ]:
df = await scan("2021-09-15 14:30")
os.makedirs("/tmp/byVolume/0915")
for row in query(df).to_records():
    code = row["code"]
    cs = Candlestick()
    await cs.plot(code, "2021-09-16 15:00", save_to="/tmp/byVolume/0915")